In [153]:
import requests
import os
import pandas as pd
import numpy as np
import json
import time

# Set up Database

In [280]:
## Make the database function
import sqlite3
from sqlite3 import Error


def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection


In [281]:
## connect to db

connection = create_connection("places_db")

Connection to SQLite DB successful


## GET data from APIs

In [80]:
## Set standard variables

cities = [("ottawa", "45.421532,-75.697189")]
pois = ["restaurant", "bar", "museum", "park", "game"]
radius = 1000

In [81]:
## Foursquare search function

def foursquareSearch(ll, query, radius):
    """
    Takes a string of ll, query and categories strings and returns the number of bars within 5km, max 50 results.
    i.e. "11111111,1111111", "query", "radius"
    """
    
    foursquare_key = os.environ["FOURSQUARE_API_KEY"]

    fsq_headers = {
    "Accept": "application/json",
    "Authorization": foursquare_key
    }

    fsq_url = "https://api.foursquare.com/v3/places/search"

    bar_params = {
        'll' : str(ll),
        'query': query,
        'radius': radius,
        'limit': '50'
    }
    
    response = requests.get(fsq_url, headers=fsq_headers, params=bar_params)
    # Check for the status code 200, to make sure if it breaks
    return response



In [82]:
# Get Ottawa fsq results
fsq_dict = {}

for city in cities:
    city_name = city[0]
    city_geo  = city[1]
    for poi in pois:
        result = foursquareSearch(city_geo, poi, str(radius))
        temp_key = "fsq_"+city_name+"_"+poi
        fsq_dict[temp_key] = result

        ## call the json, create DF, w empty list


In [83]:
## FSQ number of results

len(pd.json_normalize(fsq_dict["fsq_ottawa_restaurant"].json()['results']))

40

In [84]:
## FSQ info

pd.json_normalize(fsq_dict['fsq_ottawa_restaurant'].json()['results']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   fsq_id                        40 non-null     object 
 1   categories                    40 non-null     object 
 2   chains                        40 non-null     object 
 3   distance                      40 non-null     int64  
 4   link                          40 non-null     object 
 5   name                          40 non-null     object 
 6   timezone                      40 non-null     object 
 7   geocodes.main.latitude        40 non-null     float64
 8   geocodes.main.longitude       40 non-null     float64
 9   geocodes.roof.latitude        40 non-null     float64
 10  geocodes.roof.longitude       40 non-null     float64
 11  location.address              40 non-null     object 
 12  location.country              40 non-null     object 
 13  locatio

In [85]:
## Yelp Search Function

dec_ottawa = (45.421532, -75.697189)

def yelpSearch(dec_ottawa, query, radius):
    """
    Accepts search term string, lat & lon as floats, radius as int
    """

    yelp_key = os.environ["YELP_API_KEY"]

    yelp_headers = {
        "Accept": "application/json",
        "Authorization": "Bearer " + yelp_key
    }
    
    yelp_url = "https://api.yelp.com/v3/businesses/search"

    yelp_params = {
        "term": query,
        "latitude": dec_ottawa[0],
        "longitude": dec_ottawa[1],
        "radius": radius,
        "limit": 50
    }

    response = requests.get(yelp_url, params=yelp_params, headers=yelp_headers)

    return response

In [86]:
## Get a Yelp result

yelp_dict = {}

for city in cities:
    city_name = city[0]
    for poi in pois:
        result = yelpSearch(dec_ottawa, poi, radius)
        temp_key = "yelp_"+city_name+"_"+poi
        yelp_dict[temp_key] = result



In [87]:
len(yelp_dict['yelp_ottawa_restaurant'].json()['businesses'])

50

In [424]:
pd.json_normalize(yelp_dict['yelp_ottawa_restaurant'].json()['businesses']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        50 non-null     object 
 1   alias                     50 non-null     object 
 2   name                      50 non-null     object 
 3   image_url                 50 non-null     object 
 4   is_closed                 50 non-null     bool   
 5   url                       50 non-null     object 
 6   review_count              50 non-null     int64  
 7   categories                50 non-null     object 
 8   rating                    50 non-null     float64
 9   transactions              50 non-null     object 
 10  price                     41 non-null     object 
 11  phone                     50 non-null     object 
 12  display_phone             50 non-null     object 
 13  distance                  50 non-null     float64
 14  coordinates.

In [449]:
## Get Google Results
google_directions_key = os.environ["GOOGLE_DIRECTIONS_API_KEY"]

google_ottawa_ll = "45.421532, -75.697189"



def gplaceSearch(ll, query, radius):
    """
    Takes lat & long with coma, a query and a radius, returns a list of responses
    """

    ## set paramaters
    google_places_key = os.environ["GOOGLE_PLACES_API_KEY"]

    google_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    gplace_params = {
        "key": google_places_key,
        "location": ll,
        "radius": radius,
        "keyword": query
    }

    ## Get initial response
    response_list = []
    response = (requests.get(google_url, params=gplace_params))
    response_list.append(response.json())

    ## Check for next page to be added to response list
    try:
        next_page_token = response.json()['next_page_token']
        time.sleep(10)
        while len(next_page_token) > 10:
        # while next_page_token:

            ## New request to next page with token
            next_page_params = {
                "pagetoken": next_page_token,
                "key": google_places_key
            }
            next_response = requests.get(google_url, params=next_page_params)
            response_list.append(next_response.json())

            time.sleep(10)

            ## Assign New Token, if doesn't exist, return
            try:
                next_page_token = next_response.json()['next_page_token']
            except KeyError:
                return response_list
    except KeyError:
        pass
    return response_list



In [450]:
## Get Google results

google_dict = {}

for city in cities:
    for poi in pois:
        result = gplaceSearch(google_ottawa_ll, poi, radius)
        temp_key = "google_"+city_name+"_"+poi
        google_dict[temp_key] = result

# Organize results

### FSQ

In [426]:
## Turn FSQ results to DFs

fsq_df_dict = {}

for key in fsq_dict:
    fsq_df_dict[key] = pd.DataFrame(fsq_dict[key].json()['results'])

In [427]:
## FSQ Label each table

for df in fsq_df_dict:
    fsq_df_dict[df]['query'] = df

In [428]:
## Concat tables together

fsq_df_list = []

for df in fsq_df_dict:
    fsq_df_list.append(fsq_df_dict[df])

fsq_df = pd.concat(fsq_df_list, ignore_index=True)
fsq_df

,fsq_id,categories,chains,distance,geocodes,link,location,name,related_places,timezone,query
0,595b8ea4033693555bc7e917,"[{'id': 13354, 'name': 'Theme Restaurant', 'ic...",[],37,"{'main': {'latitude': 45.421387, 'longitude': ...",/v3/places/595b8ea4033693555bc7e917,"{'address': '116 Albert St', 'country': 'CA', ...",Mad Radish,{},America/Toronto,fsq_ottawa_restaurant
1,51a62aab498e2744fde0b125,"[{'id': 13054, 'name': 'Food Truck', 'icon': {...",[],130,"{'main': {'latitude': 45.420967, 'longitude': ...",/v3/places/51a62aab498e2744fde0b125,"{'address': 'Queen St', 'country': 'CA', 'cros...",Ottawa Streat Gourmet,{'children': [{'fsq_id': '5a39e1040868a250a3f8...,America/Toronto,fsq_ottawa_restaurant
2,5797de16498e74839d164af2,"[{'id': 13310, 'name': 'Modern European Restau...",[],197,"{'main': {'latitude': 45.42322, 'longitude': -...",/v3/places/5797de16498e74839d164af2,"{'address': '62 Sparks St', 'country': 'CA', '...",Riviera,{},America/Toronto,fsq_ottawa_restaurant
3,5467e70b498e40bd82e793a9,"[{'id': 13065, 'name': 'Restaurant', 'icon': {...",[],191,"{'main': {'latitude': 45.421786, 'longitude': ...",/v3/places/5467e70b498e40bd82e793a9,"{'address': '156 Sparks St', 'country': 'CA', ...",Bier Markt,{},America/Toronto,fsq_ottawa_restaurant
4,4b0586def964a5204f7222e3,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",[],233,"{'main': {'latitude': 45.423348, 'longitude': ...",/v3/places/4b0586def964a5204f7222e3,"{'address': '44 Sparks St', 'country': 'CA', '...",D'Arcy McGee's Irish Pub,{},America/Toronto,fsq_ottawa_restaurant
...,...,...,...,...,...,...,...,...,...,...,...
160,5f2bdac38118736f2af1069a,"[{'id': 17135, 'name': 'Toy / Game Store', 'ic...",[],575,"{'main': {'latitude': 45.425593, 'longitude': ...",/v3/places/5f2bdac38118736f2af1069a,"{'address': '50 Rideau St', 'country': 'CA', '...",The LEGO® Store CF Rideau Centre,{},America/Toronto,fsq_ottawa_game
161,57ad004b498e7db2dd96c932,"[{'id': 17135, 'name': 'Toy / Game Store', 'ic...",[],631,"{'main': {'latitude': 45.425292, 'longitude': ...",/v3/places/57ad004b498e7db2dd96c932,"{'address': '50 Rideau St', 'country': 'CA', '...",Disney,{'parent': {'fsq_id': '4afc9f14f964a520782422e...,America/Toronto,fsq_ottawa_game
162,5dd88e808afd9700073383bf,"[{'id': 17135, 'name': 'Toy / Game Store', 'ic...",[],679,"{'main': {'latitude': 45.425648, 'longitude': ...",/v3/places/5dd88e808afd9700073383bf,"{'address': '50 Rideau St', 'country': 'CA', '...",Lego,{'parent': {'fsq_id': '4afc9f14f964a520782422e...,America/Toronto,fsq_ottawa_game
163,4b16a577f964a5205bbb23e3,"[{'id': 17023, 'name': 'Computers and Electron...",[{'id': 'd515b090-d890-0132-61d3-7a163eb2a6fc'...,612,"{'main': {'latitude': 45.425775, 'longitude': ...",/v3/places/4b16a577f964a5205bbb23e3,"{'address': '50 Rideau St', 'address_extended'...",The Source,{'parent': {'fsq_id': '4afc9f14f964a520782422e...,America/Toronto,fsq_ottawa_game


### Yelp

In [429]:
## Turn yelp results to DFs

yelp_df_dict = {}

for key in yelp_dict:
    yelp_df_dict[key] = pd.DataFrame(yelp_dict[key].json()['businesses'])


In [430]:
## yelp Label each table

for df in yelp_df_dict:
    yelp_df_dict[df]['query'] = df

In [431]:
## Yelp concat tables together

yelp_df_list = []

for df in yelp_df_dict:
    yelp_df_list.append(yelp_df_dict[df])

yelp_df = pd.concat(yelp_df_list, ignore_index=True)

### Google

In [605]:
## Turn google results into DFs

google_df_dict = {}

for dict in google_dict:
    google_df_dict[dict] = []
    n = 0
    for page in google_dict[dict]:
        google_df_dict[dict].append(pd.DataFrame(google_dict[dict][n]['results']))
        n += 1

0
1
2
0
1
2
0
0
0


In [607]:
## Concat pages into one df

for pages in google_df_dict:
    google_df_dict[pages] = pd.concat(google_df_dict[pages], ignore_index=True)

In [608]:
## Label the new DFs
for df in google_df_dict:
    google_df_dict[df]['query'] = df


In [609]:
## Concat Google together

google_df_list = []

for df in google_df_dict:
    google_df_list.append(google_df_dict[df])

google_df = pd.concat(google_df_list, ignore_index=True)

In [618]:
google_df['name'].value_counts()

Level One Game Pub                          2
Prohibition Public House                    2
The Grand Pizzeria and Bar                  2
Madisons Restaurant & Bar                   2
Social Restaurant + Lounge                  2
                                           ..
Zak's Cantina                               1
Oz Kafe, ByWard Market, Ottawa              1
Trofí Restaurant                            1
The Keg Steakhouse + Bar - Ottawa Market    1
Hobbiesville                                1
Name: name, Length: 149, dtype: int64

# Best Rankings

### FSQ Rankings details

In [306]:
fsq_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   fsq_id          200 non-null    object
 1   categories      200 non-null    object
 2   chains          200 non-null    object
 3   distance        200 non-null    int64 
 4   geocodes        200 non-null    object
 5   link            200 non-null    object
 6   location        200 non-null    object
 7   name            200 non-null    object
 8   related_places  200 non-null    object
 9   timezone        200 non-null    object
 10  query           200 non-null    object
dtypes: int64(1), object(10)
memory usage: 17.3+ KB


In [309]:
id_list = list(fsq_df['fsq_id'])


['595b8ea4033693555bc7e917',
 '51a62aab498e2744fde0b125',
 '5797de16498e74839d164af2',
 '5467e70b498e40bd82e793a9',
 '4b0586def964a5204f7222e3',
 '5739fdb2498ea29b0ec70f01',
 '4b2ea641f964a52048e424e3',
 '4b68c421f964a520bc8b2be3',
 '4b9fa935f964a520993237e3',
 '4b0a9f40f964a520af2523e3',
 '4b0586e1f964a520047322e3',
 '4b0586e1f964a520077322e3',
 '4ca92d1997c8a1cdbfa58ca5',
 '54e7e183498e9af95a66b8ad',
 '4d4ae93e5129a35dffb548ac',
 '5005ecd7e4b0645daf340728',
 '572298b4498e3c813f524b4e',
 '51ae12bf498ef6174670b621',
 '4b0586dff964a520757222e3',
 '4b0ff1b1f964a5208e6623e3',
 '4b05c692f964a520fce222e3',
 '500753ede4b0e7f7beec583e',
 '51e494d4498ec07b6d8989eb',
 '59372e23f1fdaf456f0c9409',
 '4edd2e0361af8a14b9a4b0e3',
 '4b5f5841f964a52042b529e3',
 '4c268297c11dc9b673232924',
 '4d5c80b31e43236a2dd91a83',
 '4c39e37f93db0f4729172492',
 '4b0586e1f964a520f97222e3',
 '4b0586dff964a5207a7222e3',
 '5762cfca498eac69f891202f',
 '596978b8b3c9615cca41dde5',
 '4b47dc64f964a520e94126e3',
 '4b1ae780f964

In [379]:
### function to find fsq details

def fsqDetails(id_list):
    base_url='https://api.foursquare.com/v3/places/'

    foursquare_key = os.environ["FOURSQUARE_API_KEY"]

    fsq_headers = {
        "Accept": "application/json",
        "Authorization": foursquare_key
    }

    detail_params = {
        "fields": "rating,popularity,stats,price"
    }

    fsq_details_dict = {}
    for id in id_list:
        response = requests.get(base_url+id, params=detail_params, headers=fsq_headers)
        fsq_details_dict[id] = response
    return fsq_details_dict


In [ ]:
fsq_details_dict = fsqDetails(id_list)

In [381]:
print(len(id_list))
len(fsq_details_dict)

200


40

In [623]:
yelp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             151 non-null    object 
 1   alias          151 non-null    object 
 2   name           151 non-null    object 
 3   image_url      151 non-null    object 
 4   is_closed      151 non-null    bool   
 5   url            151 non-null    object 
 6   review_count   151 non-null    int64  
 7   categories     151 non-null    object 
 8   rating         151 non-null    float64
 9   coordinates    151 non-null    object 
 10  transactions   151 non-null    object 
 11  price          93 non-null     object 
 12  location       151 non-null    object 
 13  phone          151 non-null    object 
 14  display_phone  151 non-null    object 
 15  distance       151 non-null    float64
 16  query          151 non-null    object 
dtypes: bool(1), float64(2), int64(1), object(13)
memory us

In [622]:
## Clean unused data and remove duplicate ids

google_df
clean_google_df = google_df.drop_duplicates('place_id', ignore_index=True).drop(['types', 'business_status','permanently_closed', 'vicinity','icon', 'icon_mask_base_uri', 'photos', 'icon_background_color', 'opening_hours', 'place_id', 'plus_code', 'reference', 'scope'], axis=1)
clean_google_df['name'].value_counts()

El Camino                         2
JOEY Rideau                       1
The Royal Oak - Kent at Slater    1
ARC Lounge                        1
Level One Game Pub                1
                                 ..
Saigon Restaurant                 1
Ricardo's Pizza                   1
Sansotei Ramen                    1
The Whalesbone Bank Street        1
Hobbiesville                      1
Name: name, Length: 149, dtype: int64

In [624]:
## Clean unused data and remove duplicate ids

yelp_df
clean_yelp_df = yelp_df.drop_duplicates('id', ignore_index=True).drop(['id', 'alias', 'image_url', 'is_closed', 'url', 'categories', 'transactions', 'phone', 'display_phone', 'distance', 'location'], axis=1)
clean_yelp_df['name'].value_counts()

The Royal Oak                              2
Riviera                                    1
Ottawa International Animation Festival    1
Maman                                      1
Paul's Boat Lines                          1
                                          ..
The Rabbit Hole                            1
Chateau Lafayette                          1
The Gilmour                                1
Union Local 613                            1
Pub 101                                    1
Name: name, Length: 150, dtype: int64

In [625]:
# Change google column order

google_cols = clean_google_df.columns.tolist()
google_cols

new_order = [1, 4, 3, 0, 2, 5]
google_cols = [google_cols[i] for i in new_order]
google_cols

['name', 'user_ratings_total', 'rating', 'geometry', 'price_level', 'query']

In [626]:
# Re-order google dataframe

clean_google_df = clean_google_df[google_cols]
clean_google_df

,name,user_ratings_total,rating,geometry,price_level,query
0,JOEY Rideau,2670,4.3,"{'location': {'lat': 45.426424, 'lng': -75.690...",2.0,google_ottawa_restaurant
1,Bistro Ristoro,221,4.8,"{'location': {'lat': 45.4283826, 'lng': -75.69...",NaN,google_ottawa_restaurant
2,Chez Lucien,1833,4.6,"{'location': {'lat': 45.4303983, 'lng': -75.69...",2.0,google_ottawa_restaurant
3,Grill 41,237,3.6,"{'location': {'lat': 45.421695, 'lng': -75.693...",2.0,google_ottawa_restaurant
4,Norca Restaurant & Bar,98,4.6,"{'location': {'lat': 45.4256047, 'lng': -75.68...",NaN,google_ottawa_restaurant
...,...,...,...,...,...,...
145,Fandom II,295,4.1,"{'location': {'lat': 45.4204381, 'lng': -75.69...",NaN,google_ottawa_game
146,Strategy Games,83,4.5,"{'location': {'lat': 45.4161793, 'lng': -75.69...",NaN,google_ottawa_game
147,EBX,458,4.2,"{'location': {'lat': 45.4259107, 'lng': -75.69...",NaN,google_ottawa_game
148,Escape Manor - Downtown,782,4.7,"{'location': {'lat': 45.4208374, 'lng': -75.70...",NaN,google_ottawa_game


In [627]:
# Rename google cols
clean_google_df = clean_google_df.rename(columns={
                                                'user_ratings_total': 'review_count',
                                                'price_level': 'price'
                                                })
clean_google_df['name'].value_counts()

El Camino                         2
JOEY Rideau                       1
The Royal Oak - Kent at Slater    1
ARC Lounge                        1
Level One Game Pub                1
                                 ..
Saigon Restaurant                 1
Ricardo's Pizza                   1
Sansotei Ramen                    1
The Whalesbone Bank Street        1
Hobbiesville                      1
Name: name, Length: 149, dtype: int64

In [628]:
yelp_cols = clean_yelp_df.columns.tolist()
yelp_cols

['name', 'review_count', 'rating', 'coordinates', 'price', 'query']

In [629]:
## merge both dataframes together

mergeddf = pd.concat([clean_google_df, clean_yelp_df])
mergeddf

,name,review_count,rating,geometry,price,query,coordinates
0,JOEY Rideau,2670,4.3,"{'location': {'lat': 45.426424, 'lng': -75.690...",2.0,google_ottawa_restaurant,NaN
1,Bistro Ristoro,221,4.8,"{'location': {'lat': 45.4283826, 'lng': -75.69...",NaN,google_ottawa_restaurant,NaN
2,Chez Lucien,1833,4.6,"{'location': {'lat': 45.4303983, 'lng': -75.69...",2.0,google_ottawa_restaurant,NaN
3,Grill 41,237,3.6,"{'location': {'lat': 45.421695, 'lng': -75.693...",2.0,google_ottawa_restaurant,NaN
4,Norca Restaurant & Bar,98,4.6,"{'location': {'lat': 45.4256047, 'lng': -75.68...",NaN,google_ottawa_restaurant,NaN
...,...,...,...,...,...,...,...
146,St Louis Bar and Grill,42,3.5,NaN,$$,yelp_ottawa_game,"{'latitude': 45.4150151092377, 'longitude': -7..."
147,The Earl of Sussex Pub,20,3.5,NaN,$$,yelp_ottawa_game,"{'latitude': 45.428977, 'longitude': -75.6961212}"
148,Apple Store,20,3.0,NaN,$$$,yelp_ottawa_game,"{'latitude': 45.4255819, 'longitude': -75.6925..."
149,The Clocktower Brew Pub,42,3.5,NaN,$$,yelp_ottawa_game,"{'latitude': 45.42942, 'longitude': -75.69284}"


In [630]:
mergeddf.dtypes

name             object
review_count      int64
rating          float64
geometry         object
price            object
query            object
coordinates      object
dtype: object

# Start Ranking

In [631]:
mergeddf = mergeddf.drop('geometry', axis=1)

In [632]:
# set up filters

fgoogle = mergeddf['query'].str.contains('google')
fyelp = mergeddf['query'].str.contains('yelp')

f100google = (mergeddf['query'].str.contains('google')) & (mergeddf['review_count'] > mergeddf['review_count'].mean())
f100yelp = (mergeddf['query'].str.contains('yelp')) & (mergeddf['review_count'] > mergeddf['review_count'][fyelp].mean())

fgame = mergeddf['query'].str.contains('game')

games_mean = mergeddf[fgame]['review_count'].mean()

f100google_games = ((mergeddf['query'] == 'google_ottawa_game')
                    & (mergeddf['query'].str.contains('google'))
                    )
f100yelp_games = ((mergeddf['query'] == 'yelp_ottawa_game') 
                    & (mergeddf['query'].str.contains('yelp'))
                    )

In [633]:
mergeddf[f100google].sort_values(by=['query','rating'], ascending=False)[0:10].drop('coordinates', axis=1)

,name,review_count,rating,price,query
53,Sansotei Ramen,2845,4.7,1.0,google_ottawa_restaurant
2,Chez Lucien,1833,4.6,2.0,google_ottawa_restaurant
11,Riviera Ottawa,1044,4.6,3.0,google_ottawa_restaurant
26,Bite Burger House,1095,4.6,2.0,google_ottawa_restaurant
27,Town,543,4.6,3.0,google_ottawa_restaurant
36,Play Food & Wine,1017,4.6,3.0,google_ottawa_restaurant
54,The Whalesbone Bank Street,513,4.6,3.0,google_ottawa_restaurant
8,Restaurant e18hteen,491,4.5,NaN,google_ottawa_restaurant
25,Fairouz Cafe,831,4.5,3.0,google_ottawa_restaurant
38,Mamma Teresa Ristorante,1024,4.5,3.0,google_ottawa_restaurant


In [634]:
mergeddf[f100yelp].sort_values(by=['query','rating'], ascending=False)[0:10].drop('coordinates', axis=1)

,name,review_count,rating,price,query
0,Riviera,103,4.5,$$$,yelp_ottawa_restaurant
2,Sansotei Ramen,316,4.5,$$,yelp_ottawa_restaurant
6,Town,151,4.5,$$$,yelp_ottawa_restaurant
15,Shanghai Wonton Noodle,66,4.5,$,yelp_ottawa_restaurant
17,Corazón De Maíz,93,4.5,$,yelp_ottawa_restaurant
27,North & Navy,68,4.5,$$$,yelp_ottawa_restaurant
31,Roast 'n' Curries,51,4.5,$$,yelp_ottawa_restaurant
37,Benny's Bistro,113,4.5,$$,yelp_ottawa_restaurant
38,Bread & Sons Bakery,79,4.5,$,yelp_ottawa_restaurant
43,La Bottega Nicastro Fine Food Shop,158,4.5,$$,yelp_ottawa_restaurant


In [635]:
mergeddf[f100google_games].sort_values(by=['query','rating'], ascending=False)[0:10].drop('coordinates', axis=1)

,name,review_count,rating,price,query
148,Escape Manor - Downtown,782,4.7,NaN,google_ottawa_game
146,Strategy Games,83,4.5,NaN,google_ottawa_game
149,Hobbiesville,596,4.5,NaN,google_ottawa_game
147,EBX,458,4.2,NaN,google_ottawa_game
145,Fandom II,295,4.1,NaN,google_ottawa_game


In [636]:
mergeddf[f100yelp_games].sort_values(by=['query','rating'], ascending=False)[0:10].drop('coordinates', axis=1)

,name,review_count,rating,price,query
122,One In Ten Inc,1,5.0,NaN,yelp_ottawa_game
132,Capital City Derby Dolls,2,5.0,NaN,yelp_ottawa_game
133,Tick Tock Escape,1,5.0,NaN,yelp_ottawa_game
140,Dahl House Studios,1,5.0,NaN,yelp_ottawa_game
143,Canada Day Celebration Festival,8,5.0,NaN,yelp_ottawa_game
118,Escape Manor,30,4.5,NaN,yelp_ottawa_game
121,Mystery Motel Escape Room,6,4.5,NaN,yelp_ottawa_game
126,Rideau Curling Club,4,4.5,NaN,yelp_ottawa_game
127,Amaze: Ottawa Escape Game,3,4.5,NaN,yelp_ottawa_game
129,Yelp's Got Game @The Loft Board Game Lounge,2,4.5,NaN,yelp_ottawa_game


In [637]:
frestaurant = mergeddf['query'].str.contains('restaurant')
fbar = mergeddf['query'].str.contains('bar')
fmuseum = mergeddf['query'].str.contains('museum')
fpark = mergeddf['query'].str.contains('park')
fquery = mergeddf['query'].str.contains('game')

In [640]:
mergeddf.groupby(by='query').mean().sort_values(by=['query', 'review_count'], ascending=[True, False])

,review_count,rating
query,,
google_ottawa_bar,601.327273,4.145455
google_ottawa_game,442.800000,4.400000
google_ottawa_museum,1651.764706,4.305882
google_ottawa_park,429.923077,3.961538
google_ottawa_restaurant,976.783333,4.326667
yelp_ottawa_bar,38.725000,3.712500
yelp_ottawa_game,12.470588,3.779412
yelp_ottawa_museum,23.400000,3.933333
yelp_ottawa_park,10.333333,3.958333


In [639]:
mergeddf.groupby(by='query').count()['name'].sort_values(ascending=False)

query
google_ottawa_restaurant    60
google_ottawa_bar           55
yelp_ottawa_restaurant      50
yelp_ottawa_bar             40
yelp_ottawa_game            34
google_ottawa_museum        17
yelp_ottawa_museum          15
google_ottawa_park          13
yelp_ottawa_park            12
google_ottawa_game           5
Name: name, dtype: int64